## Obtendo as relações entre as disciplinas

Atualmente, no csv que contém as informações das disciplinas, não conseguimos visualizar as relações entre as disciplinas de modo tão simples, geralmente isso é feito por meio de uma [matriz de adjacência](https://pt.wikipedia.org/wiki/Matriz_de_adjac%C3%AAncia), sendo assim, esse script se propõe a gerar um csv que tenha esse formato.

## Importando as dependências

Para conseguir manipular os dados de maneira eficiente, utilizando funções auxiliares que já existem, 
usamos o pandas e o numpy, após configurá-los em sua máquina, é necessário importá-los.

In [7]:
import pandas as pd
import numpy as np

## Leitura dos dados

O trecho de código abaixo lê o arquivo csv, e o converte no formato de dataframe.

In [8]:
df = pd.read_csv('../data/disciplinas-ppc-2017.csv')
df.head()

codigo                                               nome    categoria  \
0  1411311  Fundamentos de Matemática para Ciência da Comp...  Obrigatório   
1  1411167                                      Programação I  Obrigatório   
2  1411180                       Laboratório de Programação I  Obrigatório   
3  1411174                            Introdução à Computação  Obrigatório   
4  1411312  Fundamentos de Matemática para Ciência da Comp...  Obrigatório   

   creditos pre_requisitos periodo quantidade_vagas  areas  \
0         4              -       1         120, 130    NaN   
1         4              -       1         110, 120    NaN   
2         4              -       1         140, 120    NaN   
3         4              -       1          105, 96    NaN   
4         4        1411311       2         100, 100    NaN   

                                               dicas  sigla  
0  https://github.com/OpenDevUFCG/Tamburetei/tree...  FMCC1  
1  https://github.com/OpenDevUFCG/Tamburetei/tree...     P1  
2  https://github.com/OpenDevUFCG/Tamburetei/tree...    LP1  
3  https://github.com/OpenDevUFCG/Tamburetei/tree...     IC  
4  https://github.com/OpenDevUFCG/Tamburetei/tree...  FMCC2

## Obtendo a lista de pré-requisitos

Primeiro vamos obter cada elemento da lista de pré-requistos, transformanfo-os em colunas, fazemos isso, separadamente, usando a função `apply` do pandas, que aplica a função passada como argumento, a cada valor da coluna especificada, no nosso caso, `pre_requisitos`.

Como aplicamos a função `pd.Series`, ele cria uma estrutura semelhante a uma lista em python, que representará a lista de colunas do novo dataframe.

In [9]:
arestas = df.pre_requisitos.apply(lambda x: pd.Series(np.nan if str(x) == "-" else str(x).split(",")))
arestas.head(10)

0         1    2
0      NaN       NaN  NaN
1      NaN       NaN  NaN
2      NaN       NaN  NaN
3      NaN       NaN  NaN
4  1411311       NaN  NaN
5  1411311       NaN  NaN
6  1411167   1411180  NaN
7  1411167   1411180  NaN
8  1411312       NaN  NaN
9  1109103       NaN  NaN

## Conectando os dados

No passo anterior, conseguimos fazer os elementos da lista de pre-requisitos serem colunas no dataframe de `arestas`, 
precisamos conectar esses valores ao nosso dataframe das disciplinas, para fazer isso usamos 
a operação `merge`, usando os índices de cada dataframe, como identificador para que o pandas
consiga mesclá-los.


In [10]:
arestas = arestas.merge(df, right_index = True, left_index = True)
arestas.head(8)

0         1    2   codigo  \
0      NaN       NaN  NaN  1411311   
1      NaN       NaN  NaN  1411167   
2      NaN       NaN  NaN  1411180   
3      NaN       NaN  NaN  1411174   
4  1411311       NaN  NaN  1411312   
5  1411311       NaN  NaN  1109103   
6  1411167   1411180  NaN  1411168   
7  1411167   1411180  NaN  1411181   

                                                nome    categoria  creditos  \
0  Fundamentos de Matemática para Ciência da Comp...  Obrigatório         4   
1                                      Programação I  Obrigatório         4   
2                       Laboratório de Programação I  Obrigatório         4   
3                            Introdução à Computação  Obrigatório         4   
4  Fundamentos de Matemática para Ciência da Comp...  Obrigatório         4   
5                  Cálculo Diferencial e Integral I  Obrigatório         4   
6                                     Programação II  Obrigatório         4   
7                      Laboratório de Programação II  Obrigatório         4   

     pre_requisitos periodo quantidade_vagas  areas  \
0                 -       1         120, 130    NaN   
1                 -       1         110, 120    NaN   
2                 -       1         140, 120    NaN   
3                 -       1          105, 96    NaN   
4           1411311       2         100, 100    NaN   
5           1411311       2          20, 160    NaN   
6  1411167, 1411180       2          95, 100    NaN   
7  1411167, 1411180       2          105, 90    NaN   

                                               dicas     sigla  
0  https://github.com/OpenDevUFCG/Tamburetei/tree...     FMCC1  
1  https://github.com/OpenDevUFCG/Tamburetei/tree...        P1  
2  https://github.com/OpenDevUFCG/Tamburetei/tree...       LP1  
3  https://github.com/OpenDevUFCG/Tamburetei/tree...        IC  
4  https://github.com/OpenDevUFCG/Tamburetei/tree...     FMCC2  
5  https://github.com/OpenDevUFCG/Tamburetei/tree...  CALCULO1  
6  https://github.com/OpenDevUFCG/Tamburetei/tree...        P2  
7  https://github.com/OpenDevUFCG/Tamburetei/tree...       LP2

## Removendo dados desnecessários

Agora que o formato do dataframe está quase como desejamos, podemos remover as colunas que não estamos interessados.

In [14]:
arestas = arestas.drop(list(set(df) - set(['codigo'])), axis = 1)
arestas.head(8)

0         1    2   codigo
0      NaN       NaN  NaN  1411311
1      NaN       NaN  NaN  1411167
2      NaN       NaN  NaN  1411180
3      NaN       NaN  NaN  1411174
4  1411311       NaN  NaN  1411312
5  1411311       NaN  NaN  1109103
6  1411167   1411180  NaN  1411168
7  1411167   1411180  NaN  1411181

## Separando os pré-requisitos

Como estamos interessados, em ter para cada disciplina, seu pré-requisito e se ele tiver, mais de um criamos uma nova linha, 
usamos a função `melt`.

In [15]:
arestas = arestas.melt(id_vars = ['codigo'], value_name = 'pre_requisito')
arestas.head()

codigo variable pre_requisito
0  1411311        0           NaN
1  1411167        0           NaN
2  1411180        0           NaN
3  1411174        0           NaN
4  1411312        0       1411311

## Removendo informações desnecessárias

Do dataframe acima, temos a coluna variable criada automaticamente pelo melt que não agrega nenhum valor ao que queremos, sendo assim, iremos removê-la

In [ ]:
arestas = arestas.drop('variable', axis = 1)

In [ ]:
arestas = arestas.dropna()

## Exportando os dados

Como os dados estão no formato que queremos, já podemos exportá-los, no formato csv

In [ ]:
arestas.columns = ['cod_disciplina', 'cod_pre_requisito']
arestas.to_csv('../data/relacionamentos-disciplinas-ppc2017.csv', index=False)